In [1]:
%matplotlib inline

## 使用“torch.autograd”自动微分

在训练神经网络时，最常用的算法是
**反向传播**。 在这个算法中，参数（模型权重）是
根据损失函数的**梯度**进行调整
到给定的参数。

为了计算这些梯度，PyTorch 有一个内置的微分引擎
称为“torch.autograd”。 它支持任何梯度的自动计算
计算图。

考虑最简单的一层神经网络，输入为“x”，
参数 `w` 和 `b`，以及一些损失函数。 它可以定义在
PyTorch 使用以下方式：

In [2]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## 张量、函数和计算图

此代码定义了以下**计算图**：

![该图显示了一个具有两个参数 'w' 和 'b' 的计算图，用于计算损失的梯度。](./img/computational-graph.png)

在这个网络中，`w` 和 `b` 是**参数**，我们需要
优化。因此，我们需要能够计算损失的梯度
与这些变量相关的函数。为了做到这一点，我们设置
这些张量的 `requires_grad` 属性。

> **注意：** 您可以在创建张量时设置 `requires_grad` 的值，或者稍后使用 `x.requires_grad_(True)` 方法。

我们应用于张量以构建计算图的函数是
实际上是“Function”类的对象。这个对象知道如何
计算 *forward* 方向的函数，以及如何计算
它在*反向传播*步骤期间的导数。参考
反向传播函数存储在一个的 `grad_fn` 属性中
张量。您可以在 [在
文档]（https://pytorch.org/docs/stable/autograd.html#function）。

In [3]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f93e0962520>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7f93e0962280>


## 计算梯度

为了优化神经网络中参数的权重，我们需要
计算我们的损失函数关于参数的导数，
即，我们需要 $\frac{\partial loss}{\partial w}$ 和
$\frac{\partial loss}{\partial b}$ 在一些固定值下
`x` 和 `y`。 为了计算这些导数，我们调用
`loss.backward()`，然后从 `w.grad` 和
`b.grad`：

In [4]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1498, 0.0901, 0.2149],
        [0.1498, 0.0901, 0.2149],
        [0.1498, 0.0901, 0.2149],
        [0.1498, 0.0901, 0.2149],
        [0.1498, 0.0901, 0.2149]])
tensor([0.1498, 0.0901, 0.2149])


> **注意：** 我们只能获取计算图的叶节点的 `grad` 属性，其 `requires_grad` 属性设置为 ``True``。 对于我们图中的所有其他节点，渐变将不可用。 此外，出于性能原因，我们只能在给定的图上使用“向后”一次执行梯度计算。 如果我们需要在同一个图上进行多次“向后”调用，我们需要将“retain_graph=True”传递给“向后”调用。

## 禁用梯度跟踪

默认情况下，所有带有 `requires_grad=True` 的张量都在跟踪它们的
计算历史并支持梯度计算。 然而，有
在某些情况下我们不需要这样做，例如，当我们有
训练了模型，只想将其应用于某些输入数据，即我们
只想通过网络进行*转发*计算。 我们可以停下来
通过围绕我们的计算代码来跟踪计算
`torch.no_grad()` 块：

In [5]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Another way to achieve the same result is to use the ``detach()`` method
on the tensor:




In [6]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


There are reasons you might want to disable gradient tracking:
  - To mark some parameters in your neural network at **frozen parameters**. This is
    a very common scenario for
    [fine tuning a pre-trained network](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)
  - To **speed up computations** when you are only doing forward pass, because computations on tensors that do
    not track gradients would be more efficient.



More on Computational Graphs
----------------------------
Conceptually, autograd keeps a record of data (tensors) and all executed
operations (along with the resulting new tensors) in a directed acyclic
graph (DAG) consisting of
[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
objects. In this DAG, leaves are the input tensors, roots are the output
tensors. By tracing this graph from roots to leaves, you can
automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:

- run the requested operation to compute a resulting tensor
- maintain the operation’s *gradient function* in the DAG.

The backward pass kicks off when `.backward()` is called on the DAG
root. `autograd` then:

- computes the gradients from each `.grad_fn`,
- accumulates them in the respective tensor’s `.grad` attribute
- using the chain rule, propagates all the way to the leaf tensors.

**DAGs are dynamic in PyTorch**

  An important thing to note is that the graph is recreated from scratch; after each
  `.backward()` call, autograd starts populating a new graph. This is
  exactly what allows you to use control flow statements in your model;
  you can change the shape, size and operations at every iteration if
  needed.



## 张量梯度和雅可比积

在很多情况下，我们有一个标量损失函数，我们需要计算
关于某些参数的梯度。不过也有案例
当输出函数是任意张量时。在这种情况下，PyTorch
允许您计算所谓的**雅各比积**，而不是实际的
坡度。

对于向量函数 $\vec{y}=f(\vec{x})$，其中
$\vec{x}=\langle x_1,\dots,x_n\rangle$ 和
$\vec{y}=\langle y_1,\dots,y_m\rangle$，梯度为
$\vec{y}$ 相对于 $\vec{x}$ 由 **Jacobian 给出
矩阵**：

\begin{align}\begin{align}J=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}\end{align}

PyTorch 不是计算雅可比矩阵本身，而是允许您
为给定的输入向量计算 **Jacobian Product** $v^T\cdot J$
$v=(v_1 \dots v_m)$。这是通过调用 `backward` 来实现的
$v$ 作为参数。 $v$ 的大小应该与
原始张量的大小，我们想要
计算产品：




In [7]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


请注意，当我们第二次调用 `backward` 时
参数，梯度的值是不同的。 发生这种情况是因为
在进行“向后”传播时，PyTorch **累积
梯度**，即计算梯度的值被添加到
计算图所有叶节点的`grad`属性。 如果你想
要计算适当的梯度，您需要将 `grad` 归零
之前的财产。 在现实生活中的训练中，*优化器* 可以帮助我们做到
这个。


> **注意：** 之前我们调用的是不带参数的 `backward()` 函数。 这等效于调用“backward(torch.tensor(1.0))”，这是在标量值函数（例如神经网络训练期间的损失）的情况下计算梯度的有用方法。



--------------


